# XGBOOST for classification and Hyperparameter using Optuna

In [4]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.4 MB/s eta 0:00:00


In [5]:
#importing libraries

import optuna
import xgboost as xgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [6]:
#loading data
breast_cancer = load_breast_cancer()

In [7]:
#Describe the dataset
print(breast_cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [8]:
#print the features of the dataset
print(breast_cancer.feature_names)

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [9]:
#print the target of the dataset
print(breast_cancer.target_names)

['malignant' 'benign']


In [14]:
X = breast_cancer.data
y = breast_cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
#printing the shapes of the train and test sets
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (455, 30)
X_test shape:  (114, 30)
y_train shape:  (455,)
y_test shape:  (114,)


In [ ]:
#fit a XGBoost Classifier with default parameters
clf_xgb = xgb.XGBClassifier()
clf_xgb.fit(X_train, y_train)

In [17]:
#predict the test set
y_pred = clf_xgb.predict(X_test)

#calculate the performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("Default XGBoost Classifier Performance:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"ROC-AUC: {roc_auc:.2f}")

Default XGBoost Classifier Performance:
Accuracy: 0.96
Precision: 0.96
Recall: 0.97
F1-Score: 0.97
ROC-AUC: 0.95


# Hyperparameter using Optuna

In [40]:
# Define a study object and specify the hyperparameters to optimize (e.g., learning rate, max depth, number of trees, etc.).
# Create an objective function that trains and evaluates the XGBoost classifier with the specified hyperparameters.
# Run the Optuna optimization process to find the best set of hyperparameters.
# Report the best hyperparameters and the corresponding evaluation metrics (accuracy, precision, recall, F1-score, and ROC-AUC) on the testing data.
# use your best judgment to decide which hyperparameters to optimize and what range of values to search over, as well as the evaluation metrics to use.
# use a random seed for reproducibility.


#Enter your code here
##########################################################
def objective_function(trial):
  max_depth = trial.suggest_int('max_depth',1,30)
  learning_rate = trial.suggest_uniform('learning_rate',0,0.1)
  seed = trial.suggest_int('seed',1,100)
  gamma = trial.suggest_uniform('gamma',0,0.5)
  n_estimators = trial.suggest_int('n_estimators',1000,2000)
  subsample = trial.suggest_uniform('subsample', 0.5, 1)
  min_child_weight = trial.suggest_int('min_child_weight', 1,10)
  #objective = trial.suggest_categorical('objective',['binary:logistic','binary:logitraw','binary:hinge','reg:gamma'])
  objective = trial.suggest_categorical('objective',['binary:logistic',None])
  model = xgb.XGBClassifier(learning_rate = learning_rate,
                            n_estimators = n_estimators,
                            max_depth = max_depth,
                            gamma = gamma,
                            subsample = subsample,
                            objective = objective,
                            min_child_weight = min_child_weight,
                            seed = seed
  )

  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test,y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  roc_auc = roc_auc_score(y_test, y_pred)
  return accuracy,precision,recall , f1 , roc_auc

xboostStudy = optuna.create_study(directions = ['maximize','maximize','maximize','maximize','maximize'])
xboostStudy.optimize(objective_function,n_trials = 100)

max_accuracy = 0
trial_number = 0
for i in xboostStudy.best_trials :
  if max_accuracy < i.values[0] :
    trial_number = i.number
    max_accuracy = i.values[0]


print("Best trial and it's values are", xboostStudy.best_trials[trial_number])

##########################################################

[I 2023-10-15 18:11:37,225] A new study created in memory with name: no-name-cdb7939c-2bca-4b41-b055-c4f04eb13f1e
<ipython-input-40-83c915ebedf0>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  learning_rate = trial.suggest_uniform('learning_rate',0,0.1)
<ipython-input-40-83c915ebedf0>:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  gamma = trial.suggest_uniform('gamma',0,0.5)
<ipython-input-40-83c915ebedf0>:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1)
[I 2023-10-15 18:11:37,766] Trial 0 finished with 

Best trial and it's values are FrozenTrial(number=29, state=TrialState.COMPLETE, values=[0.9824561403508771, 0.9726027397260274, 1.0, 0.9861111111111112, 0.9767441860465117], datetime_start=datetime.datetime(2023, 10, 15, 18, 12, 0, 841391), datetime_complete=datetime.datetime(2023, 10, 15, 18, 12, 1, 247228), params={'max_depth': 11, 'learning_rate': 0.05259974736341472, 'seed': 97, 'gamma': 0.38873734151692363, 'n_estimators': 1993, 'subsample': 0.7906791594068943, 'min_child_weight': 10, 'objective': 'binary:logistic'}, user_attrs={}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, distributions={'max_depth': IntDistribution(high=30, log=False, low=1, step=1), 'learning_rate': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'seed': IntDistribution(high=100, log=False, low=1, step=1), 'gamma': FloatDistribution(high=0.5, log=False, low=0.0, step=None), 'n_estimators': IntDistribution(high=2000, log=False, low=1000, step=1), 'subsample': FloatDistribution(hig